In [ ]:
# !pip install efficientnet-pytorch
import sys
sys.path = [
    '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path
sys.path = [
    '../input/ttach-kaggle/ttach/',
] + sys.path
from efficientnet_pytorch import EfficientNet


import numpy as np 
import pandas as pd 
from collections import Counter
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torchvision.models as models
import torchvision.transforms as T
import torch.nn.functional as F
import torch.nn as nn
import json
import torch
import torchvision
import cv2
import PIL
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch()

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def get_image(path):
    img = Image.open(path)
    return img


In [ ]:
class GetDataset(Dataset):
    def __init__(self, df, data_root, transforms = None, output_label = True):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.data_root = data_root
        self.transforms = transforms
        self.output_label = output_label
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):    #enforces index to be int
            
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        img = get_image(path)         
            
        #if transforms exist then apply transforms
        if self.transforms:
            img = self.transforms(img)
            
        #if label exists then get label and return
        if self.output_label:
            label = self.df.iloc[index]['label']
            return img, label
        else:
            return img

In [ ]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")
    
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        for x in self.dl:
            yield to_device(x, self.device)
            
    def __len__(self):
        return len(self.dl)

device = get_device()
device

In [ ]:
def accuracy(out, labels):
    _, preds = torch.max(out, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {"val_loss": loss.detach(), "val_acc": acc}
    
    def validation_epoch_end(self, outputs):
        batch_loss = [x["val_loss"] for x in outputs]
        epoch_loss = torch.stack(batch_loss).mean()
        batch_acc = [x["val_acc"] for x in outputs]
        epoch_acc = torch.stack(batch_acc).mean()
        return {"val_loss": epoch_loss.item(), "val_acc": epoch_acc.item()}
    
    def epoch_end(self, epoch, epochs, result):
        print("Epoch: [{}/{}], last_lr: {:.6f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
        epoch, epochs, result["lrs"][-1], result["train_loss"], result["val_loss"], result["val_acc"]))

In [ ]:
class Classifier(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = EfficientNet.from_name('efficientnet-b4')
        number_of_features =  self.network._fc.in_features
        self.network._fc = nn.Linear(number_of_features, 5)
        
    def forward(self, xb):
        return self.network(xb)
        
    def freeze(self):
        for param in self.network.parameters():
            param.requires_grad=False
        for param in self.network._fc.parameters():
            param.requires_grad=True
        
    def unfreeze(self):
        for param in self.network.parameters():
            param.requires_grad=True


In [ ]:
model = torch.load('../input/cassava-leaf-disease-detection-efficientnet-b5/mod.pth', map_location=torch.device(device) )

for parameter in model.parameters():
    parameter.requires_grad = False

model.eval()

In [ ]:
BATCH_SIZE = 128
IMG_SIZE = 512 
IMG_SHAPE = (IMG_SIZE, IMG_SIZE)
TEST_DIR = '../input/cassava-leaf-disease-classification/test_images'

test_transforms = T.Compose([
    T.Resize(IMG_SHAPE),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

])

test_images = os.listdir(TEST_DIR)
test_csv = pd.DataFrame()
test_csv["image_id"] = test_images

test_ds = GetDataset(
    test_csv,
    TEST_DIR,
    transforms = test_transforms,
    output_label = False
)

test = torch.utils.data.DataLoader(
    test_ds,
    batch_size = BATCH_SIZE,
    num_workers = 2,
    shuffle = False,
    pin_memory = False
)

test = DeviceDataLoader(test, device)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []

    for i, (images) in tk0:
        preds = []
        
        model.eval()
        with torch.no_grad():
            y_preds = model(images)
        preds.append(y_preds.softmax(1).to('cpu').numpy())
        
        probs.append(preds)
    
    probs = np.concatenate(probs, axis=1)
    print("predictions shape : ", probs.shape)
    return probs

predictions = inference(model, test, device)
test_csv['label'] = predictions.argmax(2).reshape((len(test_images)))
test_csv[['image_id', 'label']].to_csv('./submission.csv', index=False)
print(test_csv)
